In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)')
os.getcwd()

'/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)'

## 1. 결측치처리
**해당 노트북**
+ 전처리방법2-3 + y 결측치를 예측하기 + validation set 만들기

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### 1.1. 전처리방법2 데이터 가져오기

In [ ]:
train = pd.read_parquet('./data/train_preprocess_2.parquet')
# test = pd.read_parquet('./test.parquet')
test = pd.read_parquet('./data/test_preprocess_2.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())

(1000000, 19)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64


### 1.2.남은 결측치 처리 - 최빈값

In [ ]:
train.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                      11883
Carrier_Code(IATA)          108990
Carrier_ID(DOT)              11883
Tail_Number                      0
Delay                       744999
dtype: int64

In [ ]:
# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체합니다
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

Done.


In [ ]:
# 질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: # train에 없는 label인 경우
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.


In [ ]:
train.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time         0
Estimated_Arrival_Time           0
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)               0
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

### 1.3. y값 결측 데이터 추출

In [ ]:
# 레이블이 없는 데이터들을 따로 추출합니다.
null_train = train[train['Delay'].isnull()].copy()
null_train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,600.0,1900.0,0,0,252,13851,34,159,12191,42,419.0,23,10,19393.0,4319,None
1,TRAIN_000001,8,15,740.0,1024.0,0,0,256,13930,11,331,14869,45,1250.0,22,8,20304.0,310,None
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,74,11057,31,204,12953,30,544.0,3,0,19805.0,140,None
3,TRAIN_000003,7,10,905.0,1735.0,0,0,195,12892,4,119,11618,28,2454.0,26,8,19977.0,3021,None
4,TRAIN_000004,1,11,900.0,1019.0,0,0,322,14771,4,7,10157,4,250.0,22,8,20304.0,556,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,256,13930,11,270,14100,36,678.0,26,8,19977.0,2477,None
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,122,11637,32,242,13487,21,223.0,22,3,20304.0,2294,None
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,248,13796,4,159,12191,42,1642.0,23,10,19393.0,994,None
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,45,10693,41,22,10397,8,214.0,9,3,19790.0,6207,None


### 1.4. 결측없는 데이터 추출

In [ ]:
# 레이블이 없는 데이터들을 삭제합니다.
train = train.dropna()
print(train.shape)

# Delay -> 1, Not_Delayed -> 0
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

train.head()

(255001, 19)
Done.


<ipython-input-18-e34446cc7f68>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
5,TRAIN_000005,4,13,1545.0,1900.0,0,0,119,11618,28,93,11278,47,199.0,21,8,20452.0,3435,Not_Delayed,0
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,119,11618,28,47,10721,19,200.0,26,8,19977.0,3495,Not_Delayed,0
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,59,10821,18,74,11057,31,361.0,23,10,19393.0,4083,Not_Delayed,0
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,93,11278,47,277,14122,36,204.0,21,0,20452.0,241,Delayed,1
12,TRAIN_000012,1,12,1015.0,1145.0,0,0,72,11042,33,94,11292,5,1201.0,23,10,19393.0,5171,Not_Delayed,0


### 1.5. train, validation 나누기

In [ ]:
from sklearn.model_selection import train_test_split
train1, val1 = train_test_split(train, random_state=42, test_size=0.2)
print(train.shape, train1.shape, val1.shape)

(255001, 20) (204000, 20) (51001, 20)


## 2. 준지도학습진행

### 2.1. y값 예측모델 구축1
+ lgbm 이용해서 y값 예측
    + 기존에 좋은 예측 정확도가 나왔던 {'learning_rate': 0.075, 'max_depth': 8, 'n_estimators': 400, 'num_leaves': 35} 모형 사용

In [ ]:
from lightgbm import LGBMClassifier

# train1 데이터 이용헤서 모델 적합
train_x = train1.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train1['Delay_num']
null_train_x = null_train.drop(columns=['ID','Delay'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42, learning_rate=0.075, max_depth=8, n_estimators=400, num_leaves=35)

# 모델 fit
lgbm.fit(train_x, train_y)

# 예측
pred_y = lgbm.predict_proba(null_train_x)
pred_y # [Not Delayed, Delayed]


array([[0.78759941, 0.21240059],
       [0.82024937, 0.17975063],
       [0.74940414, 0.25059586],
       ...,
       [0.86956703, 0.13043297],
       [0.86077257, 0.13922743],
       [0.87743468, 0.12256532]])

In [ ]:
null_train_x['NotDelayed'] = pred_y[:,0]
null_train_x['Delayed'] = pred_y[:,1]
null_train_x[null_train_x['NotDelayed']>0.9]

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,NotDelayed,Delayed
9,6,6,650.0,838.0,0,0,207,12992,3,169,12266,42,374.0,13,8,20366.0,468,0.902670,0.097330
11,3,18,600.0,748.0,0,0,270,14100,36,103,11433,20,453.0,9,3,19790.0,1859,0.948978,0.051022
20,7,2,620.0,1900.0,0,0,59,10821,18,47,10721,19,369.0,23,10,19393.0,1369,0.924412,0.075588
26,8,26,644.0,805.0,0,0,57,10792,30,47,10721,19,395.0,18,2,20409.0,1445,0.942085,0.057915
29,10,20,800.0,1900.0,0,0,320,14747,48,18,10299,1,1448.0,1,1,19930.0,2389,0.911935,0.088065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999966,9,18,600.0,740.0,0,0,319,14730,15,93,11278,47,474.0,21,8,20452.0,2317,0.927813,0.072187
999974,4,4,600.0,859.0,0,0,322,14771,4,195,12892,4,337.0,9,3,19790.0,1939,0.914929,0.085071
999976,10,9,600.0,715.0,0,0,135,11884,48,320,14747,48,224.0,17,1,19687.0,2330,0.922331,0.077669
999978,12,6,725.0,833.0,0,0,169,12266,42,243,13495,16,305.0,26,8,19977.0,3907,0.948020,0.051980


### 2.2. 임계치 이상의 y값들을 0으로 예측


In [ ]:
# 0.9 이상인 것들 지연 안됨으로 예측
semi_super_train2 = null_train[null_train_x['NotDelayed']>0.95]
semi_super_train2['Delay_num'] = 0
semi_super_train2


# 예측값 중 max index 찾아 semi_super_train의 y값으로 넣어주기
# max_prob_idx = np.argmax(pred_y, axis=1)
# print(max_prob_idx)
# from collections import Counter
# print('불연착/연착 분포 : ',Counter(max_prob_idx))
# semi_super_train['Delay_num'] = max_prob_idx
# semi_super_train

### 2.3. 기존 데이터와 준지도학습 데이터 합치기

In [ ]:
# 데이터 합치기
semi_super_train = pd.concat([train1, semi_super_train2])
semi_super_train.sort_index(inplace=True)
semi_super_train


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
5,TRAIN_000005,4,13,1545.0,1900.0,0,0,119,11618,28,93,11278,47,199.0,21,8,20452.0,3435,Not_Delayed,0
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,119,11618,28,47,10721,19,200.0,26,8,19977.0,3495,Not_Delayed,0
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,59,10821,18,74,11057,31,361.0,23,10,19393.0,4083,Not_Delayed,0
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,93,11278,47,277,14122,36,204.0,21,0,20452.0,241,Delayed,1
13,TRAIN_000013,9,19,615.0,706.0,0,0,215,13158,42,94,11292,5,563.0,22,8,20304.0,579,Not_Delayed,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999962,TRAIN_999962,10,11,600.0,2003.0,0,0,310,14683,42,256,13930,11,1041.0,22,8,20304.0,488,Not_Delayed,0
999963,TRAIN_999963,5,2,1759.0,1926.0,0,0,204,12953,30,93,11278,47,214.0,21,3,20452.0,5204,Delayed,1
999969,TRAIN_999969,10,10,940.0,1056.0,0,0,223,13256,42,169,12266,42,316.0,19,8,20378.0,5350,Delayed,1
999985,TRAIN_999985,8,8,1914.0,2039.0,0,0,296,14492,31,183,12451,7,407.0,14,4,20436.0,1499,Not_Delayed,0


### 2.4. 준지도학습 결과 validation

+ 준지도 없는 경우 validation score
```
Extra Trees Classifier: Log Loss = 0.5017
Random Forest Classifier: Log Loss = 0.4745
Light Gradient Boosting Machine: Log Loss = 0.4426
Decision Tree Classifier: Log Loss = 10.4178
Gradient Boosting Classifier: Log Loss = 0.4478
Ada Boost Classifier: Log Loss = 0.6823
Logistic Regression: Log Loss = 0.4600
```

+ semi_super_train1 : 0.9
```
Extra Trees Classifier: Log Loss = 0.6420
Random Forest Classifier: Log Loss = 0.5825
Light Gradient Boosting Machine: Log Loss = 0.4521
Decision Tree Classifier: Log Loss = 9.7620
Gradient Boosting Classifier: Log Loss = 0.4571
Ada Boost Classifier: Log Loss = 0.6793
Logistic Regression: Log Loss = 0.4754
```
+ semi_super_train2 : 0.95
```
Extra Trees Classifier: Log Loss = 0.5257
Random Forest Classifier: Log Loss = 0.4900
Light Gradient Boosting Machine: Log Loss = 0.4432
Decision Tree Classifier: Log Loss = 10.2206
Gradient Boosting Classifier: Log Loss = 0.4482
Ada Boost Classifier: Log Loss = 0.6818
Logistic Regression: Log Loss = 0.4607
```


+ semi_super_train3 : 0.96


In [ ]:
# train, semi_super_train, val의 shape 파악

print(train1.shape, semi_super_train.shape, val1.shape)

(204000, 20) (218642, 20) (51001, 20)


In [ ]:
# 1. validation data 모델에 적합하게 바꿔주기 (val1)

# val_x & val_y

val_x = val1.drop(columns=['ID', 'Delay', 'Delay_num'])
val_y =  val1['Delay_num']


# 2. 준지도학습 경우 데이터 적합하게 바꿔주기

## 2.1. 준지도학습한 경우 (train1 + 준지도)
# semi_train_x & semi_train_y
semi_train_x = semi_super_train.drop(columns=['ID', 'Delay', 'Delay_num'])
semi_train_y =  semi_super_train['Delay_num']

## 2.2. 준지도학습 없이 기존 데이터 (train1)
# train_x &  train_y

train_x = train1.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train1['Delay_num']




print(val_x.shape, semi_train_x.shape, train_x.shape)

(51001, 17) (218642, 17) (204000, 17)


In [ ]:
# 1. semi 의 경우 validation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


# 모델 정의
models = {
    "Extra Trees Classifier": ExtraTreesClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "Light Gradient Boosting Machine": LGBMClassifier(random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42),
    "Ada Boost Classifier": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
}

# 각 모델의 성능을 비교
for name, model in models.items():
    model.fit(semi_train_x, semi_train_y)
    y_pred = model.predict_proba(val_x)
    loss = log_loss(val_y, y_pred)
    print(f"{name}: Log Loss = {loss:.4f}")

Extra Trees Classifier: Log Loss = 0.5257
Random Forest Classifier: Log Loss = 0.4900
Light Gradient Boosting Machine: Log Loss = 0.4432
Decision Tree Classifier: Log Loss = 10.2206
Gradient Boosting Classifier: Log Loss = 0.4482
Ada Boost Classifier: Log Loss = 0.6818
Logistic Regression: Log Loss = 0.4607


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 2. 기존 데이터 validation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


# 모델 정의
models = {
    "Extra Trees Classifier": ExtraTreesClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "Light Gradient Boosting Machine": LGBMClassifier(random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42),
    "Ada Boost Classifier": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
}

# 각 모델의 성능을 비교
for name, model in models.items():
    model.fit(train_x, train_y)
    y_pred = model.predict_proba(val_x)
    loss = log_loss(val_y, y_pred)
    print(f"{name}: Log Loss = {loss:.4f}")

Extra Trees Classifier: Log Loss = 0.5017
Random Forest Classifier: Log Loss = 0.4745
Light Gradient Boosting Machine: Log Loss = 0.4426
Decision Tree Classifier: Log Loss = 10.4178
Gradient Boosting Classifier: Log Loss = 0.4478
Ada Boost Classifier: Log Loss = 0.6823
Logistic Regression: Log Loss = 0.4600


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 3. 최선의 결과로 준지도학습 데이터 생성

### 전처리방법5 저장

In [ ]:
save_idx = input('몇 번째 전처리 방법인지 정수-정수를 입력하세요 : ')
train_save_name = 'train_preprocess_' + save_idx
test_save_name = 'test_preprocess_' + save_idx
train.to_parquet(f'./data/{train_save_name}.parquet')
test.to_parquet(f'./data/{test_save_name}.parquet')

몇 번째 전처리 방법인지 정수-정수를 입력하세요 : 4
